In [7]:
import re
import pandas as pd
import MySQLdb
import sys
from datetime import datetime
import numpy as np
import operator
#update this with the path to where your config file with database credentials lives
sys.path.append("D:/DataBaseUpdate/To_Clone")
from ConfigFiles import config_second as config #this reads in one's database credentials

### Master QA Script ###

In [8]:
def connect():
    mydb = MySQLdb.connect(config.host, config.username, config.password)
    cursor = mydb.cursor()
    return cursor

### Step 1:

In [ ]:
def get_counts(db):
    '''
    params: database
    returns: dictionary mapping row counts to table
    '''
    cursor = connect()
    #the information_schema tables are out of date
    cursor.execute("show tables from " + db + ";")
    tables = [i[0] for i in cursor.fetchall()]
    tables_dict = {}
    for table in tables:
        if not table.startswith('temp'):
            cursor.execute("select count(*) from " + db + "." + table)
            tables_dict[table] = cursor.fetchall()[0][0]
    return tables_dict

In [ ]:
get_counts('patent_20171226')

In [ ]:
def check_counts(old_db, new_db, ratio):
    '''
    Old_db  reference database to which we are comparing the new database
    new_db  newly parsed database
    ratio   the expected ratio in counts between the old_db and new_db, ussually 1.1
    '''
    old_table_dict = get_counts(old_db)
    new_table_dict = get_counts(new_db)
    #list of tables that we don't need to check
    unchanging = ['cpc_subsection', 'wipo_field', 'mainclass', 'subclass', 'nber_category', 'subclass_current','nber_subcategory', 'cpc_subgroup', 'persistent_inventor_disambig']
    for table in new_table_dict.keys():
        try:
            row_count_new=new_table_dict[table]
            #we want to skip temp tables because we don't expect them to match
            if not table.startswith('temp') and not table in unchanging:
                if row_count_new <= old_table_dict[table]:
                    print "Problem: New table does not have more rows than old table ", table, ". It is ", row_count_new, "and last time it was ", old_table_dict[table]       
                elif not row_count_new <= old_table_dict[table] * ratio:
                    print "Possible Problem: Suspiciously high number of entries for ", table, ". It is ", row_count_new, "and last time it was ", old_table_dict[table]       
                elif not row_count_new > old_table_dict[table]:
                    print "Problem: No new entries for", table, ". It is ", row_count_new, "and last time it was ", old_table_dict[table]
            if table in unchanging:
                if row_count_new != old_table_dict[table]:
                    print "Problem: An unchanging table has a different number of rows. ", table, " has", row_count_new, "and last time it has ", old_table_dict[table]
        except:
            print "issue with", table
        

In [25]:
def ratio_distinct_to_total_ids(old_db, new_db):
    table_column_name_dict = {'patent':'id', 'government_interest':'patent_id',  'rawlawyer': 'lawyer_id', 'rawinventor':'inventor_id', 
                             'rawassignee':'assignee_id'}
    cursor = connect()
    for table in table_column_name_dict:
        query_old = "select count(" + table_column_name_dict[table] + "), count(distinct " + table_column_name_dict[table] +") from " + old_db +"." + table
        query_new = "select count(" + table_column_name_dict[table] + "), count(distinct " + table_column_name_dict[table] +") from " + new_db + "." + table
        cursor.execute(query_old)
        old_counts = cursor.fetchall()[0]
        old_ratio = old_counts[0]/float(old_counts[1])
        cursor.execute(query_new)
        new_counts = cursor.fetchall()[0]
        new_ratio = new_counts[0]/float(new_counts[1])
        diff = (old_ratio - new_ratio)/old_ratio
        if abs(diff)<=.1:
            print "Ratio of Distinct Ids to Total Ids From Last Time is Reasonable for " + table
        else:
            print "Problem: Ratio of Distinct Ids to Total Ids from Last Time is Not Reasonalbe for " , table, " It is now, ", str(new_ratio)," and was""last time ", str(old_ratio)

In [10]:
def count_null_blank_values(old_db, new_db):
    #first go through old tables
    cursor = connect()
    cursor.execute("show tables from " + new_db)
    new_table_names = [i[0] for i in cursor.fetchall()]
    for table in new_table_names:
        print table
        if not table.startswith('temp'):
            cursor.execute("show columns from " + new_db + "." + table)
            column_names = [i[0] for i in cursor.fetchall()]
            try:
                for column in column_names:
                    print column
                    #third count null and blank from old columns in old table3s
                    cursor.execute("select count(*) from "+ new_db + "." + table + " where " + column + " is null or " + column + " = '';")
                    null_blank_new=cursor.fetchall()[0][0]
                    cursor.execute("select count(*) from "+ old_db + "." + table + " where " + column + " is null or " + column + " = '';")
                    null_blank_old=cursor.fetchall()[0][0]
                    #we'd expect new tables to have a few more null or blank values because they have more rows
                    if not null_blank_new <= null_blank_old * 1.1:
                        print "Problem: Count of Null and Blank Values is NOT Reasonable.For ", table, column, " it is ",  null_blank_new, "and last time it was ", null_blank_old
            except:
                print "Execution error with ", table, column

In [29]:
def check_latest_date(new_db, last_date = '2017-10-03'):
    cursor = connect()
    cursor.execute("select date from " + new_db + ".patent order by date desc limit 1;")
    found_date=cursor.fetchall()[0][0]
    actual_date = datetime.strptime(last_date, '%Y-%m-%d').date()
    #date_check
    if found_date != actual_date:
        print "The latest date is, ", str(found_date), "and it should be, ", str(actual_date)
    else:
        print "date matches"

### Step 2

In [34]:
def check_data_exists(update_db):
    tables = get_counts(update_db)
    for table, count in tables.iteritems():
        if count <1000:
            print table, str(count)
    

In [101]:
def check_year_groupings(new_db):
    cursor = connect()
    cursor.execute("show tables from " + new_db + ";")
    non_patent = ['location_assignee', 'rawlocation', 'mainclass_current', 'uspc_current_to_insert', 'location_inventor','government_organization','rl_temporary_update', 'inventor','lawyer', 'assignee', 'location', 'cpc_subsection', 'wipo_field', 'mainclass', 'cpc_group', 'subclass', 'nber_category', 'subclass_current','nber_subcategory', 'cpc_subgroup', 'persistent_inventor_disambig']
    tables = [i[0] for i in cursor.fetchall() if not i[0] in non_patent and not i[0].startswith("temp")]
    restart = 25
    for table in tables:
        print table
        cursor.execute('select year(p.date) as year, count(*) from ' + new_db + "." + table + ' t left join ' + new_db +'.'+ 'patent p on t.patent_id = p.id group by year(p.date) order by year')
        patents_per_year = cursor.fetchall()
        total = sum([i[1] for i in patents_per_year])
        earliest = min([i[0] for i in patents_per_year if i[0]  is not None and i[0] != ''])
        print restart, table, earliest
        restart +=1
        full_year_list = [i for i in xrange(earliest, 2018)]
        found_year_list = [int(i[0]) for i in patents_per_year if i[0]  is not None and i[0] != '']
        if set(full_year_list) != set(found_year_list):
            print "Missing year"
            print set(full_year_list) - set(found_year_list)
        previous_year_total = 0
        for year in patents_per_year:
            if year[1] < previous_year_total:
                print "Fewer patents than in previous year",year[0], year[1], previous_year_total
            previous_year_total = year[1]
        print "-------------------------------------------"

            
            
                

In [11]:
def check_lengths(merged_db):
    cursor = connect()
    cursor.execute('use ' + merged_db)
    cursor.execute('select count(*) from detail_desc_text where length  < 65540 and length > 65530')
    cutoff = cursor.execute()[0][0]
    cursor.execute('select count(*) from detail_desc_text where length  < 65550 and length > 65540')
    higher = cursor.execute()[0][0]
    print cutoff
    print higher
    if cutoff > 2*higher:
        print "Data problem"

#### Run Check Ins

In [12]:
check_lengths('patent_20171226')

TypeError: execute() takes at least 2 arguments (1 given)

In [ ]:
check_year_groupings('patent_20171226')

In [36]:
check_data_exists('jan_upload')

assignee 0
lawyer 0
patent_govintorg 0
government_interest 0
mainclass_current 0
patent_contractawardnumber 0
location 0
nber_category 0
patent_lawyer 0
nber 0
subclass_current 0
government_organization 0
wipo 0
inventor 0
botanic 331
patent_inventor 0
cpc_group 0
location_inventor 0
location_assignee 0
patent_assignee 0
cpc_current 0
uspc_current 0


In [23]:
#slow becuase it has to run counts on all the tables
check_counts('patent_20171003', 'patent_20171226', 1.1)

Possible Problem: Suspiciously high number of entries for  lawyer . It is  167828 and last time it was  8204
issue with rl_temporary_update
issue with uspc_current_to_insert
Problem: New table does not have more rows than old table  mainclass_current . It is  511 and last time it was  511
Possible Problem: Suspiciously high number of entries for  non_inventor_applicant . It is  708100 and last time it was  643616
Problem: New table does not have more rows than old table  nber_category . It is  6 and last time it was  6
Possible Problem: Suspiciously high number of entries for  patent_lawyer . It is  7380033 and last time it was  663326
Problem: New table does not have more rows than old table  nber . It is  5105937 and last time it was  5105937
Problem: New table does not have more rows than old table  subclass_current . It is  171053 and last time it was  171053
Problem: New table does not have more rows than old table  cpc_group . It is  656 and last time it was  656
Possible Problem

In [26]:
ratio_distinct_to_total_ids('patent_20171003', 'patent_20171226')

Ratio of Distinct Ids to Total Ids From Last Time is Reasonable for patent
Ratio of Distinct Ids to Total Ids From Last Time is Reasonable for rawassignee
Ratio of Distinct Ids to Total Ids From Last Time is Reasonable for rawlawyer
Ratio of Distinct Ids to Total Ids From Last Time is Reasonable for government_interest
Ratio of Distinct Ids to Total Ids From Last Time is Reasonable for rawinventor


In [31]:
count_null_blank_values('patent_20171003', 'patent_20171226')

application
id
patent_id
type
number
country
date


C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: Warning: Incorrect date value: '' for column 'date' at row 1
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: Warning: Incorrect date value: '' for column 'date' at row 1


id_transformed
number_transformed
series_code_transformed_from_type
assignee
id
type
name_first
name_last
organization
botanic
uuid
patent_id
latin_name
variety
Problem: Count of Null and Blank Values is NOT Reasonable.For  botanic variety  it is  4 and last time it was  3
brf_sum_text
uuid
patent_id
text
claim
uuid
patent_id
text
dependent
sequence
exemplary
cpc_current
uuid
patent_id
section_id
subsection_id
group_id
subgroup_id
category
sequence
cpc_group
id
title
cpc_subgroup
id
title
cpc_subsection
id
title
detail_desc_text
uuid
patent_id
text
length


OperationalError: (1054, "Unknown column 'length' in 'where clause'")

In [30]:
check_latest_date('patent_20171226', last_date = '2017-12-26')

date matches
